# Prompt Engineering Techniques
By Nilesh Joshi

Reference: https://www.promptingguide.ai/techniques 


This notebook contains examples demonstrating different prompt engineering techniques.

- Zero shot
- Few shot
- CoT
- Self Consistency
- Generate knowledge prompting
- Prompt chaining
- Tree of thoughts
- RAG
- Automatic reasoning and tool use
- Automatic prompt engineer
- Active-prompt
- Directional Stimulus prompting
- Program aided language models
- ReAct
- Multimodal CoT
- Graph Prompting

---

In [2]:
#checking current dir
import os
cwd = os.getcwd()
cwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/aoai-poc-vm1/code/Users/nileshjoshi/AOAI-POC'

In my virtual environment, I have already installed libraries, below commands to get details of libraries installed

In [3]:
%pip show openai

Name: openai
Version: 1.10.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /anaconda/envs/condaAoai/lib/python3.11/site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip freeze

aiohttp==3.9.3
aiosignal==1.3.1
annotated-types==0.6.0
anyio==4.2.0
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
attrs==23.2.0
azure-ai-ml==1.13.0
azure-common==1.1.28
azure-core==1.29.7
azure-identity==1.15.0
azure-mgmt-core==1.4.0
azure-storage-blob==12.19.0
azure-storage-file-datalake==12.14.0
azure-storage-file-share==12.15.0
cachetools==5.3.2
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==3.3.2
colorama==0.4.6
comm @ file:///work/ci_py311/comm_1677709131612/work
cryptography==42.0.2
dataclasses-json==0.6.4
debugpy @ file:///croot/debugpy_1690905042057/work
decorator @ file:///opt/conda/conda-bld/decorator_1643638310831/work
distro==1.9.0
executing @ file:///opt/conda/conda-bld/executing_1646925071911/work
frozenlist==1.4.1
google-api-core==2.16.1
google-auth==2.27.0
googleapis-common-protos==1.62.0
greenlet==3.0.3
h11==0.14.0
httpcore==1.0.2
httpx==0.26.0
idna==3.6
ipykernel @ file:///croot/ipykernel_1705933831282/work
ipython @ file:///croot/ipython_

In [5]:
#import required libraries 
import openai
from openai import AzureOpenAI
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [6]:
#Load environment variables from env file. 
load_dotenv("credentials.env", override=True)

# API configuration
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
MODEL_DEPLOYMENT_NAME = "gpt-35-turbo"
# set variables required for LangChain
os.environ["OPENAI_API_TYPE"]=os.environ["OPENAI_API_TYPE"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_BASE"]= os.environ["AZURE_OPENAI_ENDPOINT"] # Your Azure OpenAI resource endpoint
os.environ["OPENAI_API_KEY"]= os.environ["AZURE_OPENAI_API_KEY"] # Your Azure OpenAI resource key

In [7]:
#set_openAI_params method - to set parameters for Open AI model
#Default:  'temperature=0.7' and 'top-p=1'

def set_openAI_params(
    model=MODEL_DEPLOYMENT_NAME,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

#completion method : To call AOAI API
def get_completion(params, messages):
    """ GET completion from openai api"""
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version="2023-12-01-preview" #Function calling works in this API
    )
    response = client.chat.completions.create(
        model=MODEL_DEPLOYMENT_NAME,
        messages=messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty']
    )    
    return response

Basic prompt example:

In [8]:
# basic example
params = set_openAI_params()

prompt = "The sky is"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [9]:
response.choices[0].message.content

' blue with fluffy white clouds.'

Try with different temperature to compare results:

In [10]:
params = set_openAI_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

 blue and clear, with a few fluffy white clouds scattered across the horizon. The sun is shining brightly, casting a warm glow over everything below.

**Temperature**: Controls randomness. Lowering the temperature means that the model produces more repetitive and deterministic responses. Increasing the temperature results in more unexpected or creative responses. Try adjusting temperature or Top P but not both.

**Top probabilities:** Similar to temperature, this controls randomness but uses a different method. Lowering Top P narrows the model’s token selection to likelier tokens. Increasing Top P lets the model choose from tokens with both high and low likelihood. Try adjusting temperature or Top P but not both.

Reference for more details: https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line%2Cpython&pivots=programming-language-studio#settings


### 1.1 Text Summarization

In [11]:
params = set_openAI_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Antibiotics are medications used to treat bacterial infections by either killing the bacteria or preventing their reproduction, but they are not effective against viral infections and misuse can lead to antibiotic resistance.

In [12]:
params = set_openAI_params(temperature=0.7)
prompt = """ I am 5. 
Explain the above in one sentence:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The speaker is stating their age as 5 years old.

### 1.2 Question Answering

In [13]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


Mice

Unsure about answer

### 1.3 Text Classification

In [14]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Neutral

Below is modification to same prompt to instruct the model to provide an explanation to the answer selected. 

In [16]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Provide sentiment and explaination about the same in details"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Sentiment: Neutral

Explanation: The statement "I think the food was okay" reflects a neutral sentiment as the speaker is not expressing strong positive or negative feelings about the food. They may have found it to be satisfactory but not exceptional. Overall, the sentiment is neutral as it does not convey strong emotions in either direction.

### 1.4 Role Playing

In [18]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]


response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Certainly. Black holes are formed when massive stars collapse under their own gravity at the end of their life cycle. This collapse causes the star to become extremely dense, with a gravitational pull so strong that even light cannot escape from it. This creates a region of spacetime from which no particles or radiation can escape, known as the event horizon. This phenomenon is what we observe as a black hole.

Role Playing example of using Open AI model as customer support agent. Provide necessary instructions about personality of the AI support agent

In [19]:
prompt = """You are an Xbox customer support agent whose primary goal is to help users with issues they are experiencing with their Xbox devices.
You are friendly and concise. You only provide factual answers to queries, and do not provide answers that are not related to Xbox."""

messages = [
    {
        "role": "system",
        "content": prompt
    }
]

messages = [
    {
        "role": "user",
        "content": "How do I perform factory reset? "
    }

]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Performing a factory reset on a device can vary depending on the specific make and model. Here are general steps to perform a factory reset on most devices:

1. Back up your important data: Before performing a factory reset, make sure to back up any important data, such as photos, videos, contacts, and documents to ensure you don't lose them.

2. Access the settings: Go to the "Settings" or "System" menu on your device.

3. Find the reset option: Look for an option like "Backup & reset," "Reset," or "Factory data reset" within the settings menu.

4. Initiate the reset: Select the option to perform a factory reset and confirm your decision when prompted. You may need to enter your device's PIN or passcode to proceed.

5. Wait for the process to complete: The device will then begin the factory reset process, which may take a few minutes to complete.

6. Set up your device: Once the factory reset is complete, you will need to set up your device as if it were new, including connecting to Wi-Fi, signing in to your accounts, and restoring your backed-up data.

Remember that performing a factory reset will erase all data on the device, so be sure to back up

### 1.5 Code Generation

In [22]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)


SELECT s.StudentId, s.StudentName
FROM students s
JOIN departments d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';

### 1.6 Reasoning

In [23]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The odd numbers in the group are 15, 5, 13, 7, and 1.

Adding them together:
15 + 5 + 13 + 7 + 1 = 41

The result, 41, is an odd number.

Exercise: Improve the prompt to have a better structure and output format.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [24]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

The answer is False.

### 2.3 Chain-of-Thought (CoT) Prompting

In [25]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

### 2.4 Zero-shot CoT

In [26]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

You started with 10 apples. Then you gave away 2 + 2 = 4 apples. 

So, you had 10 - 4 = 6 apples left. 

After that, you bought 5 more apples, so you had 6 + 5 = 11 apples. 

Then you ate 1, so you remained with 11 - 1 = 10 apples. 

So, you remained with 10 apples in total.

### 2.5 Self-Consistency
As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here. 


---

In [27]:
prompt = """When I was 6 my sister was half my age.Now, I am 70 how old is my sister?"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Your sister is 68 years old.

above answer is wrong, in such scenarios we need to use CoT with Fewshots (aka Self consistency). See below

In [28]:
prompt = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5

Q: When I was 6 my sister was half my age. Now I’m 70 how old is my sister?
A:"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

When you were 6, your sister was half your age, so she was 3 years old. Now that you are 70, your sister is 70 - (6-3) = 67 years old.

### 2.6 Generate Knowledge Prompting

As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here. 

In [29]:
prompt = """Part of golf is trying to get a higher point total than others. Yes or No?"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

Yes.

This type of mistake reveals the limitations of LLMs to perform tasks that require more knowledge about the world. How do we improve this with knowledge generation?

First, we generate a few "knowledges"

In [30]:
prompt = """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.

Explain and Answer: """

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

No, the objective of golf is not to get a higher point total than others. In fact, the objective of golf is to play a set of holes in the least number of strokes. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game. Therefore, the goal is to have the lowest point total, not the highest.

Thus above approach of feeding knowledge works better in such scenarios.

### 2.7 Prompt Chaining

https://www.promptingguide.ai/techniques/prompt_chaining

In [48]:
prompt = """You are a helpful assistant. Your task is to help answer a question given a document.The first step is to extract quotes relevant to the question from the document, delimited by ####. Please output the list of quotes using <quotes></quotes>. Respond with "No relevant quotes found!" if no relevant quotes were found.
####Prompt engineering is a relatively new discipline for developing and optimizing prompts to efficiently use language models (LMs) for a wide variety of applications and research topics. Prompt engineering skills help to better understand the capabilities and limitations of large language models (LLMs).
Below are some of the prompt engineering techniques:
#### Chain-of-thought (CoT) prompting ####
#### Generated knowledge prompting #### 
#### Least-to-most prompting #### Self-consistency decoding #### Complexity-based prompting #### Self-refine #### Tree-of-thought prompting#### Maieutic prompting #### Directional-stimulus prompting #### Textual inversion and embeddings #### Using gradient descent to search for prompts #### Prompt injection ####"""

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

<quotes>Chain-of-thought (CoT) prompting</quotes>
<quotes>Least-to-most prompting</quotes>
<quotes>Self-consistency decoding</quotes>
<quotes>Complexity-based prompting</quotes>
<quotes>Tree-of-thought prompting</quotes>
<quotes>Directional-stimulus prompting</quotes>
<quotes>Textual inversion and embeddings</quotes>
<quotes>Using gradient descent to search for prompts</quotes>
<quotes>Prompt injection</quotes>

Prompt chaining
Tree of thought
RAG
Automatic reasoning and tool use
Automatic prompt engineer
Active prompt

